作业：实践编程部分

情境：保险行业对话的LM

In [1]:
#数据预处理：去除分隔符和标点符号，将txt文本转成列表，取列表中的中文、英文，分别生成一个一维列表以储存全部string类型的中英文数据

corpus='C:/Users/GuoBaiPing/Desktop/个人文件/人工智能与NLP 5期/train.txt' #注意/
FILE=open(corpus,'r',encoding='UTF-8') #encoding='UTF-8'写入，防止python读取出错
lines = FILE.readlines() #readlines()函数会把txt文本每一行的回车符\n读取出来
sub_file_cn=[] #将目标文件封装进一个空列表
sub_file_en=[]

for line in lines:
    items = line.strip().split(' ++$++ ') #line.strp()旨在将\n去除。本段循环可以封装到一个def函数里
    sub_file_cn.append(items[2]) #items将每一行分成若干列
    sub_file_cn=''.join(sub_file_cn).split('？')#将sub_file_cn看做一个sequence，去除标点符号后缀合成语料
    sub_file_en.append(items[3])
    sub_file_en=' '.join(sub_file_en).split('?')


In [2]:
import collections
from collections import Counter
import jieba

cn_tokens=list(jieba.cut(sub_file_cn[0])) #中文切词
words_count=Counter(cn_tokens) #1-gram计数，生成列表

_2_gram_words = [cn_tokens [i] + cn_tokens [i+1] for i in range(len(cn_tokens)-1)] #生成2-gram的tokens
_2_gram_word_counts = Counter(_2_gram_words) #2-gram计数，生成列表

def get_1_gram_count(word):
    if word in words_count : 
        return words_count[word]
    else:
        return words_count.most_common()[-1][-1] #确保如果找不到该词，必定返回最小的计数，即1，这样在进行除法运算时不会出现分子为0的情况


def get_2_gram_count(word):
    if word in _2_gram_words : 
        return _2_gram_word_counts[word]
    else:
        return _2_gram_word_counts.most_common()[-1][-1] 
    
def _2_gram_models(sentence):
    tokens=list(jieba.cut(sentence)) #将input的语句进行切分
    probability=1 #因为是个似然函数，所以初始化为1
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word=tokens[i+1]
        pro=get_2_gram_count(word+next_word) / get_1_gram_count(word)
        probability *= pro #似然计算
    return probability

_2_gram_models('怎么提交人寿保险')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\GUOBAI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.875 seconds.
Prefix dict has been built succesfully.


0.003174603174603174

In [3]:
合理提问 = """
提问 = 代词 动作 险种 | 情境 判断 动作 险种
代词 = 怎样 | 怎么 | 如何 
动作 = 提交 | 覆盖 | 报告 | 接受 | 需要 | 逆转 | 缴交 | 有
险种 = 人寿保险 | 车险 | 残疾保险 | 健康保险 | 医疗保险 | 租赁保险
情境 = 我的车剐蹭 | 如果已经诊断重疾 | 离婚情况 | 雇主收取吸烟者费用
判断 = 是否 | 可不可以 | 能不能 | 行不行
"""

import random

#整理用于随机读取的数据库 ---- 使用字典进行存储
def get_generation(raw_string, target, stmt_split = '=', or_split = '|'):
    rules = dict() #对象为字典
    for line in raw_string.split('\n'): #与预处理的数据特点有关
        if not line: continue
        stmt, expression = line.split(stmt_split) #分开
        rules[stmt.strip()]=expression.split(or_split) # 分开的同时去掉首尾的空格，确保映射
        generated = generate (rules, target = target) # 嵌套，见下
    return generated

def generate(rule, target):
    if target in rule: 
        words = rule[target] #在字典中，取得对应的语法数据集
        word = random.choice(words) #随机选择一套语法
        return ''.join(generate(rule, target=c.strip()) for c in word.split()) #去掉首尾空格
    else:
        return target #如果映射不对，则会回到原先的语法规则

In [7]:
queries=()
q_list=[]
for i in range(8):
    question=get_generation(合理提问,target='提问')
    pro=_2_gram_models(question)
    queries=(question, pro)
    q_list.append(queries)

sort_result=sorted(q_list, key=lambda x: x[1], reverse=True)
print(sort_result[0][0])
    

怎样报告人寿保险


存在问题：
1） 提问的生成依赖文本的预处理，所以提问还是有些奇怪；
2） 需要提高概率值，但提高到怎样的概率值才不会出现过拟合现象？
3） 代码方面，还得考虑泛化……